In [ ]:
!pip install tensorflow opencv-python albumentations 

In [ ]:
# This process will capture some photos from your webcam

import os
import time
import uuid
import cv2

# If not exists create folder /data/....
images_destination_path = os.path.join('data','captured_images') 
labels_destination_path = os.path.join('data','captured_labels')
images_number = 30

cap = cv2.VideoCapture(0) # Define your cam index
for imgnum in range(images_number):
    ret, frame = cap.read()
    image_name = os.path.join(images_destination_path  , f'{str(uuid.uuid1())}.jpg') # Generate a UUID1 as image name
    
    cv2.imwrite(image_name, frame)
    cv2.imshow('frame', frame)

    time.sleep(0.5) # Time per photo
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Now open app LabelMe and add each label to captured_labels for each captured_images
# https://github.com/labelmeai/labelme

In [ ]:
import random
import shutil

image_extensions = ['.jpg'] # Define a list of image extensions like'.jpeg', '.png', '.bmp'

# Create a list of image filenames in 'DATA'
imgs_list = [filename for filename in os.listdir(images_destination_path) if os.path.splitext(filename)[-1] in image_extensions]

In [ ]:
len(imgs_list) # Check images list size

In [ ]:
# Separate data into 3 parts (Train, Validation, Test)
random.seed(42) # Sets the random seed 
random.shuffle(imgs_list) # Shuffle the list of image filenames

# Determine number of images for each part 70% - 15% - 15%
train_size = int(len(imgs_list) * 0.70)
val_size = int(len(imgs_list) * 0.15)
test_size = int(len(imgs_list) * 0.15)

In [ ]:
DATA = "data"
TRAIN = "train"
VAL = "val"
TEST = "test"
IMAGES = "images"
LABELS = "labels"

# DATA Destination folders
train_images_folder = os.path.join(DATA, TRAIN, IMAGES)
val_images_folder = os.path.join(DATA, VAL, IMAGES)
test_images_folder = os.path.join(DATA, TEST, IMAGES)

train_labels_folder = os.path.join(DATA, TRAIN, LABELS)
val_labels_folder = os.path.join(DATA, VAL, LABELS)
test_labels_folder = os.path.join(DATA, TEST, LABELS)

# Create destination folders if they don't exist (Train, Validation, Test)
for folder_path in [train_images_folder, val_images_folder, test_images_folder, train_labels_folder, val_labels_folder, test_labels_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
# Copy images to destination folders
for i, f in enumerate(imgs_list):
    if i < train_size:
        dest_folder = train_images_folder
    elif i < train_size + val_size:
        dest_folder = val_images_folder
    else:
        dest_folder = test_images_folder
    shutil.copy(os.path.join(images_destination_path, f), os.path.join(dest_folder, f))

In [ ]:
# Load each label for correct place folder
for folder in [TRAIN,TEST,VAL]:
    for file in os.listdir(os.path.join(DATA, folder, IMAGES)):
        filename = file.split('.')[0]+'.json'
        
        found_label = os.path.join(labels_destination_path, filename)
        if (os.path.exists(found_label)):
            new_label_path = os.path.join(DATA, folder, LABELS, filename)
            os.replace(found_label, new_label_path)

In [ ]:
# This albumentations library create new images, changing randomly some pixels, to improve data accuracy
# For more albumentation types -> https://demo.albumentations.ai/ 
import albumentations as alb 

augmentor = alb.Compose([
    alb.RandomCrop(width=450, height=450),
    alb.HorizontalFlip(p=0.5),
    alb.RandomBrightnessContrast(p=0.2),
    alb.RandomGamma(p=0.2),
    alb.VerticalFlip(p=0.5)
], bbox_params=alb.BboxParams(format='albumentations', label_fields=['class_labels']))

In [ ]:
# Create new folder to AUG_DATA -> This folder will containg albumentations images and labels

AUG_DATA = "aug_data" # Aug -> Augmentation

# AUG Destination folders
aug_train_images_folder = os.path.join(AUG_DATA, TRAIN, IMAGES)
aug_val_images_folder = os.path.join(AUG_DATA, VAL, IMAGES)
aug_test_images_folder = os.path.join(AUG_DATA, TEST, IMAGES)

aug_train_labels_folder = os.path.join(AUG_DATA, TRAIN, LABELS)
aug_val_labels_folder = os.path.join(AUG_DATA, VAL, LABELS)
aug_test_labels_folder = os.path.join(AUG_DATA, TEST, LABELS)

# Create AUG destination folders if they don't exist (Train, Validation, Test)
for folder_path in [aug_train_images_folder, aug_val_images_folder, aug_test_images_folder, aug_train_labels_folder, aug_val_labels_folder, aug_test_labels_folder]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [ ]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

# Running algumentation in a pipeline to all images to generate X images to each image
for partition in [TRAIN, TEST, VAL]: 
    for image in os.listdir(os.path.join(DATA, partition, IMAGES)):
        img = cv2.imread(os.path.join(DATA, partition, IMAGES, image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join(DATA, partition, 'labels', f'{image.split(".")[0]}.json')
        try: 
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    label = json.load(f)
                coords[0] = label['shapes'][0]['points'][0][0]
                coords[1] = label['shapes'][0]['points'][0][1]
                coords[2] = label['shapes'][0]['points'][1][0]
                coords[3] = label['shapes'][0]['points'][1][1]
                coords = list(np.divide(coords, [640,480,640,480])) # Define images Width/Height
        except Exception as e:
            print(e)

        try: 
            for x in range(30): # Number of algumentation images per image 
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
                cv2.imwrite(os.path.join(AUG_DATA, partition, IMAGES, f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: # If doesnt exists label, set bbox to ZEROS
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 


                with open(os.path.join(AUG_DATA, partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)